# Transform (creating time break variables) financial ration data and merging pollution tables to S3

# Objective(s)

## Business needs 

Transform (creating time-break variables and fixed effect) asif_financial_ratio data and merging pollution, industry and city mandate tables using Athena and save output to S3 + Glue. 

## Description

**Objective**

Construct the time-break and fixed effect variables

**Construction variables**

* time-break: If year prior to 2006, then False else True
* Fixed effect:
  * city-industry: FE_c_i
  * time-industry: FE_t_i
  * city-time: FE_c_t

**Steps**

We will clean the table by doing the following steps:

1. merge tables asif_city_industry_financial_ratio  with
  1. china_city_sector_pollution 
  2. china_city_tcz_spz
  3. china_city_reduction_mandate
  4. china_code_normalised
  5. ind_cic_2_name

**Cautious**

* Make sure there is no duplicates


Target
* The file is saved in S3: 
  * bucket: datalake-datascience 
  * path: DATA/ENVIRONMENT/CHINA/FYP/FINANCIAL_CONTRAINT/PAPER_FYP_FINANCE_POL/BASELINE 
* Glue data catalog should be updated
  * database: environment 
  * table prefix: fin_dep_pollution 
    * table name (prefix + last folder S3 path): fin_dep_pollution_baseline 
* Analytics
  * HTML:  ANALYTICS/HTML_OUTPUT/FIN_DEP_POLLUTION_BASELINE 
  * Notebook:  ANALYTICS/OUTPUT/FIN_DEP_POLLUTION_BASELINE 

# Metadata

* Key: jfl55vdtx66109y
* Parent key (for update parent):  
* Notebook US Parent (i.e the one to update): 
* https://github.com/thomaspernet/Financial_dependency_pollution/blob/master/01_data_preprocessing/02_transform_tables/01_fin_dep_pol_baseline.md
* Epic: Epic 2
* US: US 2
* Date Begin: 11/24/2020
* Duration Task: 1
* Description: Transform (creating time-break variables and fixed effect) asif_financial_ratio data and merging pollution, industry and city mandate tables using Athena and save output to S3 + Glue. 
* Step type: Transform table
* Status: Active
* Source URL: US 02 Baseline
* Task type: Jupyter Notebook
* Users: Thomas Pernet
* Watchers: Thomas Pernet
* User Account: https://468786073381.signin.aws.amazon.com/console
* Estimated Log points: 8
* Task tag: #baseline-table,#financial-ratio,#policy,#pollution
* Toggl Tag: #data-transformation
* current nb commits: 0
* Meetings:  
* Presentation:  
* Email Information:  
  * thread: Number of threads: 0(Default 0, to avoid display email)
  *  

# Input Cloud Storage [AWS/GCP]

## Table/file

* Origin: 
* Athena
* Name: 
* asif_city_industry_financial_ratio
* china_city_reduction_mandate
* china_city_sector_pollution 
* china_city_tcz_spz
* china_code_normalised
* ind_cic_2_name
* Github: 
  * https://github.com/thomaspernet/Financial_dependency_pollution/blob/master/01_data_preprocessing/02_transform_tables/00_asif_financial_ratio.md
  * https://github.com/thomaspernet/Financial_dependency_pollution/blob/master/01_data_preprocessing/00_download_data_from/CITY_REDUCTION_MANDATE/city_reduction_mandate.py
  * https://github.com/thomaspernet/Financial_dependency_pollution/blob/master/01_data_preprocessing/00_download_data_from/CITY_SECTOR_POLLUTION/city_sector_pollution.py
  * https://github.com/thomaspernet/Financial_dependency_pollution/blob/master/01_data_preprocessing/00_download_data_from/TCZ_SPZ/tcz_spz_policy.py
  * https://github.com/thomaspernet/Financial_dependency_pollution/blob/master/01_data_preprocessing/00_download_data_from/CITY_CODE_CORRESPONDANCE/city_code_correspondance.py
  * https://github.com/thomaspernet/Financial_dependency_pollution/blob/master/01_data_preprocessing/00_download_data_from/CIC_NAME/cic_industry_name.py

# Destination Output/Delivery

## Table/file

* Origin: 
* S3
* Athena
* Name:
* DATA/ENVIRONMENT/CHINA/FYP/FINANCIAL_CONTRAINT/PAPER_FYP_FINANCE_POL/BASELINE
* fin_dep_pollution_baseline
* GitHub:
* https://github.com/thomaspernet/Financial_dependency_pollution/blob/master/01_data_preprocessing/02_transform_tables/01_fin_dep_pol_baseline.md
* URL: 
  * datalake-datascience/DATA/ENVIRONMENT/CHINA/FYP/FINANCIAL_CONTRAINT/PAPER_FYP_FINANCE_POL/BASELINE

In [1]:
from awsPy.aws_authorization import aws_connector
from awsPy.aws_s3 import service_s3
from awsPy.aws_glue import service_glue
from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns
import os, shutil, json

path = os.getcwd()
parent_path = str(Path(path).parent.parent)


name_credential = 'financial_dep_SO2_accessKeys.csv'
region = 'eu-west-3'
bucket = 'datalake-datascience'
path_cred = "{0}/creds/{1}".format(parent_path, name_credential)

In [2]:
con = aws_connector.aws_instantiate(credential = path_cred,
                                       region = region)
client= con.client_boto()
s3 = service_s3.connect_S3(client = client,
                      bucket = bucket, verbose = True) 
glue = service_glue.connect_glue(client = client) 

In [3]:
pandas_setting = True
if pandas_setting:
    cm = sns.light_palette("green", as_cmap=True)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', None)

# Prepare query 

Write query and save the CSV back in the S3 bucket `datalake-datascience` 

# Steps

1. Load the pollution data: china_city_sector_pollution
2. Merge with:
    - asif_city_industry_financial_ratio
    - china_city_reduction_mandate
    - china_city_tcz_spz
    - china_code_normalised
    - ind_cic_2_name

## Example step by step

In [4]:
DatabaseName = 'environment'
s3_output_example = 'SQL_OUTPUT_ATHENA'

In [10]:
query= """
SELECT *
FROM environment.china_city_sector_pollution
LIMIT 10
"""
output = s3.run_query(
                    query=query,
                    database=DatabaseName,
                    s3_output=s3_output_example,
    filename = 'example_1'
                )
output

,year,prov2013,provinces,citycode,citycn,cityen,indus_code,ind2,ttoutput,twaste_water,tcod,tammonia_nitrogen,twaste_gas,tso2,tnox,tsmoke_dust,tsoot,lower_location,larger_location,coastal
0,1998,河北省,Hebei,1301,石家庄,Shijiazhuang,2110,21,800.0,800,0.0,0.0,540,8640,0,3600,0,Coastal,Eastern,Yes
1,1998,河北省,Hebei,1301,石家庄,Shijiazhuang,2720,27,346125.0,23814734,13626894.0,0.0,251746,119819,0,135020,40000,Coastal,Eastern,Yes
2,1998,河北省,Hebei,1301,石家庄,Shijiazhuang,2631,26,35206.0,6857995,1313520.0,0.0,1991,19000,0,34800,0,Coastal,Eastern,Yes
3,1998,河北省,Hebei,1301,石家庄,Shijiazhuang,2600,26,4400.0,2000,86400.0,0.0,919,192000,0,12000,0,Coastal,Eastern,Yes
4,1998,河北省,Hebei,1301,石家庄,Shijiazhuang,3541,35,11184.4,216750,32384.0,0.0,3174,79844,0,24122,0,Coastal,Eastern,Yes
5,1998,河北省,Hebei,1301,石家庄,Shijiazhuang,3124,31,449.0,49600,20000.0,0.0,1110,19360,0,3630,3600,Coastal,Eastern,Yes
6,1998,河北省,Hebei,1301,石家庄,Shijiazhuang,2651,26,22652.0,3178348,560076.0,0.0,72792,447270,0,199888,0,Coastal,Eastern,Yes
7,1998,河北省,Hebei,1301,石家庄,Shijiazhuang,3511,35,190.0,400,0.0,0.0,10,60,0,300,0,Coastal,Eastern,Yes
8,1998,河北省,Hebei,1301,石家庄,Shijiazhuang,2621,26,124696.4,26307242,12107578.0,0.0,493985,6454855,0,4783895,0,Coastal,Eastern,Yes
9,1998,河北省,Hebei,1301,石家庄,Shijiazhuang,4160,41,79863.0,1025573,124382.0,0.0,31790,267253,0,141798,0,Coastal,Eastern,Yes


Count number of observation in `china_city_sector_pollution`

In [16]:
query= """
SELECT COUNT(*) AS CNT
FROM environment.china_city_sector_pollution
"""
output = s3.run_query(
                    query=query,
                    database=DatabaseName,
                    s3_output=s3_output_example,
    filename = 'count_1'
                )
output

,CNT
0,189475


In [8]:
query= """
SELECT *
FROM firms_survey.asif_city_industry_financial_ratio
LIMIT 10
"""
output = s3.run_query(
                    query=query,
                    database=DatabaseName,
                    s3_output=s3_output_example,
    filename = 'example_2'
                )
output

,citycode,cic,year,working_capital_cit,working_capital_ci,working_capital_i,asset_tangibility_cit,asset_tangibility_ci,asset_tangibility_i,current_ratio_cit,current_ratio_ci,current_ratio_i,cash_assets_cit,cash_assets_ci,cash_assets_i,liabilities_assets_cit,liabilities_assets_ci,liabilities_assets_i,return_on_asset_cit,return_on_asset_ci,return_on_asset_i,sales_assets_cit,sales_assets_ci,sales_assets_i
0,4309,2490,2000,NaN,NaN,NaN,3178,3301.00,9948.428571,2.45018,2.33334,2.00324,NaN,NaN,NaN,0.36335,0.36335,0.51886,NaN,NaN,-0.15760,-23.39583,-5.95101,7.84145
1,4309,2490,2001,NaN,NaN,NaN,3152,3301.00,9948.428571,3.05155,2.33334,2.00324,NaN,NaN,NaN,NaN,0.36335,0.51886,NaN,NaN,-0.15760,-18.39053,-5.95101,7.84145
2,4402,1312,1998,NaN,NaN,NaN,28006,27098.00,54691.709273,0.51531,0.62995,1.27969,NaN,NaN,NaN,NaN,0.90388,0.84745,NaN,NaN,-0.00277,NaN,-0.23441,36.70418
3,4402,1312,1999,NaN,NaN,NaN,28027,27098.00,54691.709273,0.52982,0.62995,1.27969,NaN,NaN,NaN,NaN,0.90388,0.84745,NaN,NaN,-0.00277,-5.59066,-0.23441,36.70418
4,4402,1312,2000,NaN,NaN,NaN,29500,27098.00,54691.709273,0.41277,0.62995,1.27969,NaN,NaN,NaN,0.90388,0.90388,0.84745,NaN,NaN,-0.00277,-4.03443,-0.23441,36.70418
5,4402,1312,2001,NaN,NaN,NaN,22859,27098.00,54691.709273,1.06191,0.62995,1.27969,NaN,NaN,NaN,NaN,0.90388,0.84745,NaN,NaN,-0.00277,8.92187,-0.23441,36.70418
6,4402,2040,1998,NaN,8877.25,14804.161224,1550,1125.25,7989.767981,3.08824,2.93294,2.21921,NaN,-0.34685,-1.49301,NaN,0.34687,0.44941,NaN,9.18496,7.94262,NaN,-44.60324,1.30116
7,4402,2040,1999,NaN,8877.25,14804.161224,0,1125.25,7989.767981,NaN,2.93294,2.21921,NaN,-0.34685,-1.49301,NaN,0.34687,0.44941,NaN,9.18496,7.94262,0.00000,-44.60324,1.30116
8,4402,2040,2000,NaN,8877.25,14804.161224,0,1125.25,7989.767981,NaN,2.93294,2.21921,NaN,-0.34685,-1.49301,NaN,0.34687,0.44941,NaN,9.18496,7.94262,NaN,-44.60324,1.30116
9,4402,2040,2002,NaN,8877.25,14804.161224,2951,1125.25,7989.767981,3.45554,2.93294,2.21921,NaN,-0.34685,-1.49301,NaN,0.34687,0.44941,NaN,9.18496,7.94262,6.91108,-44.60324,1.30116


In [9]:
query= """
SELECT *
FROM policy.china_city_reduction_mandate
LIMIT 10
"""
output = s3.run_query(
                    query=query,
                    database=DatabaseName,
                    s3_output=s3_output_example,
    filename = 'example_3'
                )
output

,citycn,cityen,prov2013,so2_05_city_reconstructed,so2_obj_2010,tso2_mandate_c,so2_perc_reduction_c,ttoutput,in_10_000_tonnes
0,上海,Shanghai,上海市,5.121932,3.794024,1.327908,0.26,230979744.0,1.327908
1,昆明,Kunming,云南省,2.237456,2.147444,0.090013,0.02,6370188.0,0.900126
2,曲靖,Qujing,云南省,1.223688,1.174459,0.049229,0.01,1871252.8,0.492288
3,玉溪,Yuxi,云南省,0.770952,0.739937,0.031015,0.01,2821705.5,0.310153
4,思茅,Simao,云南省,0.249400,0.239366,0.010033,0.00,240283.8,0.100333
5,保山,Baoshan,云南省,0.225593,0.216518,0.009076,0.00,271308.6,0.090756
6,昭通,Zhaotong,云南省,0.172856,0.165902,0.006954,0.00,388868.7,0.069540
7,丽江,Lijiang,云南省,0.097863,0.093926,0.003937,0.00,57121.5,0.039370
8,临沧,Lincang,云南省,0.095948,0.092088,0.003860,0.00,162455.5,0.038600
9,包头,Baotou,内蒙古自治区,4.658621,4.479443,0.179178,0.01,4747308.5,1.791777


Count number of observation in `china_city_reduction_mandate`

In [21]:
query= """
SELECT COUNT(*) AS CNT
FROM policy.china_city_reduction_mandate
"""
output = s3.run_query(
                    query=query,
                    database=DatabaseName,
                    s3_output=s3_output_example,
    filename = 'count_2'
                )
output

,CNT
0,285


In [11]:
query= """
SELECT *
FROM policy.china_city_tcz_spz
LIMIT 10
"""
output = s3.run_query(
                    query=query,
                    database=DatabaseName,
                    s3_output=s3_output_example,
    filename = 'example_4'
                )
output

,province,city,geocode4_corr,tcz,spz
0,Beijing,Beijing,1101,1,1
1,Tianjin,Tianjin,1201,1,1
2,Hebei,Shijiazhuang,1301,1,1
3,Hebei,Tangshan,1302,1,0
4,Hebei,Qinhuangdao,1303,0,1
5,Hebei,Handan,1304,1,0
6,Hebei,Xingtai,1305,1,0
7,Hebei,Baoding,1306,1,1
8,Hebei,Zhangjiakou,1307,1,0
9,Hebei,Chengde,1308,1,0


In [13]:
query= """
SELECT *
FROM chinese_lookup.china_city_code_normalised
LIMIT 10
"""
output = s3.run_query(
                    query=query,
                    database=DatabaseName,
                    s3_output=s3_output_example,
    filename = 'example_5'
                )
output

,extra_code,geocode4_corr,citycn,cityen,citycn_correct,cityen_correct,province_cn,province_en
0,1100,1101,北京,Beijing,北京,Beijing,北京市,Beijing
1,1101,1101,北京,Beijing,北京,Beijing,北京市,Beijing
2,1102,1101,北京,Beijing,北京,Beijing,北京市,Beijing
3,1200,1201,天津,Tianjin,天津,Tianjin,天津市,Tianjin
4,1201,1201,天津,Tianjin,天津,Tianjin,天津市,Tianjin
5,1202,1201,天津,Tianjin,天津,Tianjin,天津市,Tianjin
6,1301,1301,石家庄,Shijiazhuang,石家庄,Shijiazhuang,河北省,Hebei
7,1302,1302,唐山,Tangshan,唐山,Tangshan,河北省,Hebei
8,1303,1303,秦皇岛,Qinhuangdao,秦皇岛,Qinhuangdao,河北省,Hebei
9,1304,1304,邯郸,Handan,邯郸,Handan,河北省,Hebei


In [14]:
query= """
SELECT *
FROM chinese_lookup.ind_cic_2_name
LIMIT 10
"""
output = s3.run_query(
                    query=query,
                    database=DatabaseName,
                    s3_output=s3_output_example,
    filename = 'example_6'
                )
output

,cic,industry_name,short
0,13,Processing of Food from Agricultural Products,Processing foods
1,14,Foods,Foods
2,15,Beverages,Beverages
3,16,Tobacco,Tobacco
4,17,Textile,Textile
5,18,"""Textile Wearing Apparel",Footwear
6,19,"""Leather",Fur
7,20,"""Processing of Timber",Manufacture of Wood
8,21,Furniture,Furniture
9,22,Paper and Paper Products,Paper


Test merge on `china_city_reduction_mandate` and `china_city_code_normalised`

Shanghai for instance has more than 3 codes

In [23]:
query = """
SELECT china_city_code_normalised.citycn, china_city_code_normalised.cityen, extra_code, geocode4_corr, tso2_mandate_c, in_10_000_tonnes
FROM policy.china_city_reduction_mandate
INNER JOIN chinese_lookup.china_city_code_normalised 
ON china_city_reduction_mandate.citycn = china_city_code_normalised.citycn
LIMIT 10
"""
output = s3.run_query(
                    query=query,
                    database=DatabaseName,
                    s3_output=s3_output_example,
    filename = 'example_7'
                )
output

,citycn,cityen,extra_code,geocode4_corr,tso2_mandate_c,in_10_000_tonnes
0,上海,Shanghai,3102,3101,1.327908,1.327908
1,上海,Shanghai,3101,3101,1.327908,1.327908
2,上海,Shanghai,3100,3101,1.327908,1.327908
3,昆明,Kunming,5301,5301,0.090013,0.900126
4,曲靖,Qujing,5303,5303,0.049229,0.492288
5,玉溪,Yuxi,5304,5304,0.031015,0.310153
6,思茅,Simao,5308,5308,0.010033,0.100333
7,保山,Baoshan,5312,5305,0.009076,0.090756
8,保山,Baoshan,5305,5305,0.009076,0.090756
9,昭通,Zhaotong,5306,5306,0.006954,0.069540


## Check the table with many city code

A city can have more than one codes over time. Below, we show which tables have more than one code so we know on which columns we need to merge `extra_code` or `geocode4_corr` 

**china_city_sector_pollution**

The table `china_city_sector_pollution` needs to be matched using `extra_code`

In [25]:
query = """
SELECT DISTINCT(citycode)
FROM environment.china_city_sector_pollution
WHERE citycode in ('3100','3101','3102')
"""
output = s3.run_query(
                    query=query,
                    database=DatabaseName,
                    s3_output=s3_output_example,
    filename = 'example_8'
                )
output

,citycode
0,3102
1,3101


**asif_city_industry_financial_ratio**

The table `asif_city_industry_financial_ratio` needs to be matched using `extra_code`

In [26]:
query = """
SELECT DISTINCT(citycode)
FROM firms_survey.asif_city_industry_financial_ratio
WHERE citycode in ('3100','3101','3102')
"""
output = s3.run_query(
                    query=query,
                    database=DatabaseName,
                    s3_output=s3_output_example,
    filename = 'example_8'
                )
output

,citycode
0,3102
1,3101
2,3100


**china_city_tcz_spz**

The table `china_city_tcz_spz` needs to be matched using `geocode4_corr`

In [28]:
query = """
SELECT DISTINCT(geocode4_corr)
FROM policy.china_city_tcz_spz
WHERE geocode4_corr in ('3100','3101','3102')
"""
output = s3.run_query(
                    query=query,
                    database=DatabaseName,
                    s3_output=s3_output_example,
    filename = 'example_8'
                )
output

,geocode4_corr
0,3101


### Test merge

Let's test the merge with the tables on `citycode` :

- china_city_sector_pollution
    - asif_city_industry_financial_ratio

to be sure there is no duplicates

In [29]:
query = """
WITH merge AS (
SELECT asif_city_industry_financial_ratio.year, asif_city_industry_financial_ratio.citycode, cic, ind2, tso2, lower_location, larger_location, coastal
FROM environment.china_city_sector_pollution
INNER JOIN firms_survey.asif_city_industry_financial_ratio
ON 
china_city_sector_pollution.citycode  = asif_city_industry_financial_ratio.citycode AND
china_city_sector_pollution.year  = asif_city_industry_financial_ratio.year AND
china_city_sector_pollution.indus_code  = asif_city_industry_financial_ratio.cic 
WHERE year in ('2001','2002', '2003', '2004', '2005', '2006', '2007')
)

SELECT CNT, COUNT(CNT) AS dup
FROM (
  SELECT citycode, year, cic, COUNT(*) AS CNT
  FROM merge
GROUP BY citycode, year, cic
  ) AS count_dup
GROUP BY CNT
"""
output = s3.run_query(
                    query=query,
                    database=DatabaseName,
                    s3_output=s3_output_example,
    filename = 'example_9'
                )
output

,CNT,dup
0,1,126942


Test by adding the  `china_city_reduction_mandate` and `china_city_code_normalised`

In [30]:
query = """
WITH merge AS (
SELECT asif_city_industry_financial_ratio.year, asif_city_industry_financial_ratio.citycode, cic, ind2, tso2,tso2_mandate_c,in_10_000_tonnes,lower_location, larger_location, coastal
FROM environment.china_city_sector_pollution
INNER JOIN firms_survey.asif_city_industry_financial_ratio
ON 
china_city_sector_pollution.citycode  = asif_city_industry_financial_ratio.citycode AND
china_city_sector_pollution.year  = asif_city_industry_financial_ratio.year AND
china_city_sector_pollution.indus_code  = asif_city_industry_financial_ratio.cic 
INNER JOIN (
  
  SELECT china_city_code_normalised.citycn, china_city_code_normalised.cityen, extra_code, geocode4_corr, tso2_mandate_c, in_10_000_tonnes
FROM policy.china_city_reduction_mandate
INNER JOIN chinese_lookup.china_city_code_normalised 
ON china_city_reduction_mandate.citycn = china_city_code_normalised.citycn
) as city_mandate
  ON china_city_sector_pollution.citycode  = city_mandate.extra_code

WHERE year in ('2001','2002', '2003', '2004', '2005', '2006', '2007')
)

SELECT CNT, COUNT(CNT) AS dup
FROM (
  SELECT citycode, year, cic, COUNT(*) AS CNT
  FROM merge
GROUP BY citycode, year, cic
  ) AS count_dup
GROUP BY CNT
"""
output = s3.run_query(
                    query=query,
                    database=DatabaseName,
                    s3_output=s3_output_example,
    filename = 'example_9'
                )
output

,CNT,dup
0,1,126942


Test by adding `china_city_tcz_spz` on `geocode4_corr`

In [31]:
query = """
WITH merge AS (
SELECT asif_city_industry_financial_ratio.year, asif_city_industry_financial_ratio.citycode,tcz, spz, cic, ind2, tso2,tso2_mandate_c,in_10_000_tonnes,lower_location, larger_location, coastal
FROM environment.china_city_sector_pollution
INNER JOIN firms_survey.asif_city_industry_financial_ratio
ON 
china_city_sector_pollution.citycode  = asif_city_industry_financial_ratio.citycode AND
china_city_sector_pollution.year  = asif_city_industry_financial_ratio.year AND
china_city_sector_pollution.indus_code  = asif_city_industry_financial_ratio.cic 
INNER JOIN (
  
  SELECT china_city_code_normalised.citycn, china_city_code_normalised.cityen, extra_code, geocode4_corr, tso2_mandate_c, in_10_000_tonnes
FROM policy.china_city_reduction_mandate
INNER JOIN chinese_lookup.china_city_code_normalised 
ON china_city_reduction_mandate.citycn = china_city_code_normalised.citycn
) as city_mandate
  ON china_city_sector_pollution.citycode  = city_mandate.extra_code

LEFT JOIN policy.china_city_tcz_spz
  ON china_city_sector_pollution.citycode  = china_city_tcz_spz.geocode4_corr
WHERE year in ('2001','2002', '2003', '2004', '2005', '2006', '2007')
)

SELECT CNT, COUNT(CNT) AS dup
FROM (
  SELECT citycode, year, cic, COUNT(*) AS CNT
  FROM merge
GROUP BY citycode, year, cic
  ) AS count_dup
GROUP BY CNT
"""
output = s3.run_query(
                    query=query,
                    database=DatabaseName,
                    s3_output=s3_output_example,
    filename = 'example_9'
                )
output

,CNT,dup
0,1,126942


# Table `fin_dep_pollution_baseline`

Since the table to create has missing value, please use the following at the top of the query

```
CREATE TABLE database.table_name WITH (format = 'PARQUET') AS
```

Choose a location in S3 to save the CSV. It is recommended to save in it the `datalake-datascience` bucket. Locate an appropriate folder in the bucket, and make sure all output have the same format

In [17]:
s3_output = 'DATA/ENVIRONMENT/CHINA/FYP/FINANCIAL_CONTRAINT/PAPER_FYP_FINANCE_POL/BASELINE'
table_name = 'fin_dep_pollution_baseline'

First, we need to delete the table (if exist)

In [ ]:
try:
    response = glue.delete_table(
        database=DatabaseName,
        table=table_name
    )
    print(response)
except Exception as e:
    print(e)

Clean up the folder with the previous csv file. Be careful, it will erase all files inside the folder

In [ ]:
s3.remove_all_bucket(path_remove = s3_output)

In [ ]:
%%time
query = """
SELECT 
  asif_city_industry_financial_ratio.year, 
  CASE WHEN asif_city_industry_financial_ratio.year in (
    '2001', '2002', '2003', '2004', '2005'
  ) THEN 'FALSE' WHEN asif_city_industry_financial_ratio.year in ('2006', '2007') THEN 'TRUE' END AS period, 
  provinces, 
  china_city_sector_pollution.cityen, 
  asif_city_industry_financial_ratio.citycode, 
  tcz, 
  spz, 
  asif_city_industry_financial_ratio.cic, 
  ind2, 
  short, 
  tso2, 
  tso2_mandate_c, 
  in_10_000_tonnes, 
  working_capital_cit, 
  working_capital_ci, 
  working_capital_i, 
  asset_tangibility_cit, 
  asset_tangibility_ci, 
  asset_tangibility_i, 
  current_ratio_cit, 
  current_ratio_ci, 
  current_ratio_i, 
  cash_assets_cit, 
  cash_assets_ci, 
  cash_assets_i, 
  liabilities_assets_cit, 
  liabilities_assets_ci, 
  liabilities_assets_i, 
  return_on_asset_cit, 
  return_on_asset_ci, 
  return_on_asset_i, 
  sales_assets_cit, 
  sales_assets_ci, 
  sales_assets_i, 
  lower_location, 
  larger_location, 
  coastal 
FROM 
  environment.china_city_sector_pollution 
  INNER JOIN firms_survey.asif_city_industry_financial_ratio ON china_city_sector_pollution.citycode = asif_city_industry_financial_ratio.citycode 
  AND china_city_sector_pollution.year = asif_city_industry_financial_ratio.year 
  AND china_city_sector_pollution.indus_code = asif_city_industry_financial_ratio.cic 
  INNER JOIN (
    SELECT 
      china_city_code_normalised.citycn, 
      china_city_code_normalised.cityen, 
      extra_code, 
      geocode4_corr, 
      tso2_mandate_c, 
      in_10_000_tonnes 
    FROM 
      policy.china_city_reduction_mandate 
      INNER JOIN chinese_lookup.china_city_code_normalised ON china_city_reduction_mandate.citycn = china_city_code_normalised.citycn
  ) as city_mandate ON china_city_sector_pollution.citycode = city_mandate.extra_code 
  LEFT JOIN policy.china_city_tcz_spz ON china_city_sector_pollution.citycode = china_city_tcz_spz.geocode4_corr 
  LEFT JOIN chinese_lookup.ind_cic_2_name ON china_city_sector_pollution.ind2 = ind_cic_2_name.cic 
WHERE 
  asif_city_industry_financial_ratio.year in (
    '2001', '2002', '2003', '2004', '2005', 
    '2006', '2007'
  ) 
LIMIT 
  10



""".format(DatabaseName, table_name)
output = s3.run_query(
                    query=query,
                    database=DatabaseName,
                    s3_output=s3_output,
                )
output

In [ ]:
query = """
SELECT COUNT(*) AS CNT
FROM {}.{} 
""".format(DatabaseName, table_name)
output = s3.run_query(
                    query=query,
                    database=DatabaseName,
                    s3_output=s3_output_example,
    filename = 'count_{}'.format(table_name)
                )
output

# Validate query

This step is mandatory to validate the query in the ETL. If you are not sure about the quality of the query, go to the next step.

To validate the query, please fillin the json below. Don't forget to change the schema so that the crawler can use it.

1. Add a partition key:
    - Inform if there is group in the table so that, the parser can compute duplicate
2. Add the steps number -> Not automtic yet. Start at 0
3. Change the schema if needed. It is highly recommanded to add comment to the fields
4. Provide a description -> detail the steps 

1. Add a partition key

In [ ]:
partition_keys = []

2. Add the steps number

In [ ]:
step = 0

3. Change the schema

Bear in mind that CSV SerDe (OpenCSVSerDe) does not support empty fields in columns defined as a numeric data type. All columns with missing values should be saved as string. 

In [ ]:
glue.get_table_information(
    database = DatabaseName,
    table = table_name)['Table']['StorageDescriptor']['Columns']

In [ ]:
schema = [
    {
        "Name": "VAR1",
        "Type": "",
        "Comment": ""
    },
    {
        "Name": "VAR2",
        "Type": "",
        "Comment": ""
    }
]

4. Provide a description

In [ ]:
description = """

"""

5. provide metadata

- DatabaseName
- TablePrefix
- 

In [ ]:
json_etl = {
    'step': 1,
    'description':description,
    'query':query,
    'schema': schema,
    'partition_keys':partition_keys,
    'metadata':{
    'DatabaseName' : DatabaseName,
    'TableName' : table_name,
    'target_S3URI' : os.path.join('s3://',bucket, s3_output),
    'from_athena': 'True'    
    }
}
json_etl

In [ ]:
with open(os.path.join(str(Path(path).parent), 'parameters_ETL_TEMPLATE.json')) as json_file:
    parameters = json.load(json_file)

Remove the step number from the current file (if exist)

In [ ]:
index_to_remove = next(
                (
                    index
                    for (index, d) in enumerate(parameters['TABLES']['PREPARATION']['STEPS'])
                    if d["step"] == step
                ),
                None,
            )
if index_to_remove != None:
    parameters['TABLES']['PREPARATION']['STEPS'].pop(index_to_remove)

In [ ]:
parameters['TABLES']['PREPARATION']['STEPS'].append(json_etl)

Save JSON

In [ ]:
with open(os.path.join(str(Path(path).parent), 'parameters_ETL_TEMPLATE.json'), "w")as outfile:
    json.dump(parameters, outfile)

# Create or update the data catalog

The query is saved in the S3 (bucket `datalake-datascience`) but the table is not available yet in the Data Catalog. Use the function `create_table_glue` to generate the table and update the catalog.

Few parameters are required:

- name_crawler: Name of the crawler
- Role: Role to temporary provide an access tho the service
- DatabaseName: Name of the database to create the table
- TablePrefix: Prefix of the table. Full name of the table will be `TablePrefix` + folder name

To update the schema, please use the following structure

```
schema = [
    {
        "Name": "VAR1",
        "Type": "",
        "Comment": ""
    },
    {
        "Name": "VAR2",
        "Type": "",
        "Comment": ""
    }
]
```

In [ ]:
glue.update_schema_table(
    database = DatabaseName,
    table = table_name,
    schema= schema)

## Check Duplicates

One of the most important step when creating a table is to check if the table contains duplicates. The cell below checks if the table generated before is empty of duplicates. The code uses the JSON file to create the query parsed in Athena. 

You are required to define the group(s) that Athena will use to compute the duplicate. For instance, your table can be grouped by COL1 and COL2 (need to be string or varchar), then pass the list ['COL1', 'COL2'] 

In [ ]:
partition_keys = []

with open(os.path.join(str(Path(path).parent), 'parameters_ETL_TEMPLATE.json')) as json_file:
    parameters = json.load(json_file)

In [ ]:
### COUNT DUPLICATES
if len(partition_keys) > 0:
    groups = ' , '.join(partition_keys)

    query_duplicates = parameters["ANALYSIS"]['COUNT_DUPLICATES']['query'].format(
                                DatabaseName,table_name,groups
                                )
    dup = s3.run_query(
                                query=query_duplicates,
                                database=DatabaseName,
                                s3_output="SQL_OUTPUT_ATHENA",
                                filename="duplicates_{}".format(table_name))
    display(dup)


# Analytics

In this part, we are providing basic summary statistic. Since we have created the tables, we can parse the schema in Glue and use our json file to automatically generates the analysis.

The cells below execute the job in the key `ANALYSIS`. You need to change the `primary_key` and `secondary_key` 

For a full analysis of the table, please use the following Lambda function. Be patient, it can takes between 5 to 30 minutes. Times varies according to the number of columns in your dataset.

Use the function as follow:

- `output_prefix`:  s3://datalake-datascience/ANALYTICS/OUTPUT/TABLE_NAME/
- `region`: region where the table is stored
- `bucket`: Name of the bucket
- `DatabaseName`: Name of the database
- `table_name`: Name of the table
- `group`: variables name to group to count the duplicates
- `primary_key`: Variable name to perform the grouping -> Only one variable for now
- `secondary_key`: Variable name to perform the secondary grouping -> Only one variable for now
- `proba`: Chi-square analysis probabilitity
- `y_var`: Continuous target variables

Check the job processing in Sagemaker: https://eu-west-3.console.aws.amazon.com/sagemaker/home?region=eu-west-3#/processing-jobs

The notebook is available: https://s3.console.aws.amazon.com/s3/buckets/datalake-datascience?region=eu-west-3&prefix=ANALYTICS/OUTPUT/&showversions=false

Please, download the notebook on your local machine, and convert it to HTML:

```
cd "/Users/thomas/Downloads/Notebook"
aws s3 cp s3://datalake-datascience/ANALYTICS/OUTPUT/asif_unzip_data_csv/Template_analysis_from_lambda-2020-11-22-08-12-20.ipynb .

## convert HTML no code
jupyter nbconvert --no-input --to html Template_analysis_from_lambda-2020-11-21-14-30-45.ipynb
jupyter nbconvert --to html Template_analysis_from_lambda-2020-11-22-08-12-20.ipynb
```

Then upload the HTML to: https://s3.console.aws.amazon.com/s3/buckets/datalake-datascience?region=eu-west-3&prefix=ANALYTICS/HTML_OUTPUT/

Add a new folder with the table name in upper case

In [ ]:
import boto3

key, secret_ = con.load_credential()
client_lambda = boto3.client(
    'lambda',
    aws_access_key_id=key,
    aws_secret_access_key=secret_,
    region_name = region)

In [ ]:
primary_key = ''
secondary_key = ''
y_var = ''

In [ ]:
payload = {
    "input_path": "s3://datalake-datascience/ANALYTICS/TEMPLATE_NOTEBOOKS/template_analysis_from_lambda.ipynb",
    "output_prefix": "s3://datalake-datascience/ANALYTICS/OUTPUT/{}/".format(table_name.upper()),
    "parameters": {
        "region": "{}".format(region),
        "bucket": "{}".format(bucket),
        "DatabaseName": "{}".format(DatabaseName),
        "table_name": "{}".format(table_name),
        "group": "{}".format(','.join(partition_keys)),
        "keys": "{},{}".format(primary_key,secondary_key),
        "y_var": "{}".format(y_var),
        "threshold":0
    },
}
payload

In [ ]:
response = client_lambda.invoke(
    FunctionName='RunNotebook',
    InvocationType='RequestResponse',
    LogType='Tail',
    Payload=json.dumps(payload),
)
response

For a partial analysis, run the cells below

In [ ]:
#table = 'XX'
schema = glue.get_table_information(
    database = DatabaseName,
    table = table_name
)['Table']
schema

## Count missing values

In [ ]:
from datetime import date
today = date.today().strftime('%Y%M%d')

In [ ]:
table_top = parameters["ANALYSIS"]["COUNT_MISSING"]["top"]
table_middle = ""
table_bottom = parameters["ANALYSIS"]["COUNT_MISSING"]["bottom"].format(
    DatabaseName, table_name
)

for key, value in enumerate(schema["StorageDescriptor"]["Columns"]):
    if key == len(schema["StorageDescriptor"]["Columns"]) - 1:

        table_middle += "{} ".format(
            parameters["ANALYSIS"]["COUNT_MISSING"]["middle"].format(value["Name"])
        )
    else:
        table_middle += "{} ,".format(
            parameters["ANALYSIS"]["COUNT_MISSING"]["middle"].format(value["Name"])
        )
query = table_top + table_middle + table_bottom
output = s3.run_query(
    query=query,
    database=DatabaseName,
    s3_output="SQL_OUTPUT_ATHENA",
    filename="count_missing",  ## Add filename to print dataframe
    destination_key=None,  ### Add destination key if need to copy output
)
display(
    output.T.rename(columns={0: "total_missing"})
    .assign(total_missing_pct=lambda x: x["total_missing"] / x.iloc[0, 0])
    .sort_values(by=["total_missing"], ascending=False)
    .style.format("{0:,.2%}", subset=["total_missing_pct"])
    .bar(subset="total_missing_pct", color=["#d65f5f"])
)

# Brief description table

In this part, we provide a brief summary statistic from the lattest jobs. For the continuous analysis with a primary/secondary key, please add the relevant variables you want to know the count and distribution

## Categorical Description

During the categorical analysis, we wil count the number of observations for a given group and for a pair.

### Count obs by group

- Index: primary group
- nb_obs: Number of observations per primary group value
- percentage: Percentage of observation per primary group value over the total number of observations

Returns the top 10 only

In [ ]:
for field in schema["StorageDescriptor"]["Columns"]:
    if field["Type"] in ["string", "object", "varchar(12)"]:

        print("Nb of obs for {}".format(field["Name"]))

        query = parameters["ANALYSIS"]["CATEGORICAL"]["PAIR"].format(
            DatabaseName, table_name, field["Name"]
        )
        output = s3.run_query(
            query=query,
            database=DatabaseName,
            s3_output="SQL_OUTPUT_ATHENA",
            filename="count_categorical_{}".format(
                field["Name"]
            ),  ## Add filename to print dataframe
            destination_key=None,  ### Add destination key if need to copy output
        )

        ### Print top 10

        display(
            (
                output.set_index([field["Name"]])
                .assign(percentage=lambda x: x["nb_obs"] / x["nb_obs"].sum())
                .sort_values("percentage", ascending=False)
                .head(10)
                .style.format("{0:.2%}", subset=["percentage"])
                .bar(subset=["percentage"], color="#d65f5f")
            )
        )

## Continuous description

There are three possibilities to show the ditribution of a continuous variables:

1. Display the percentile
2. Display the percentile, with one primary key
3. Display the percentile, with one primary key, and a secondary key

### 1. Display the percentile

- pct: Percentile [.25, .50, .75, .95, .90]

In [ ]:
table_top = ""
table_top_var = ""
table_middle = ""
table_bottom = ""

var_index = 0
size_continuous = len([len(x) for x in schema["StorageDescriptor"]["Columns"] if 
                       x['Type'] in ["float", "double", "bigint", 'int']])
cont = 0
for key, value in enumerate(schema["StorageDescriptor"]["Columns"]):
    if value["Type"] in ["float", "double", "bigint", 'int']:
        cont +=1

        if var_index == 0:
            table_top_var += "{} ,".format(value["Name"])
            table_top = parameters["ANALYSIS"]["CONTINUOUS"]["DISTRIBUTION"][
                "bottom"
            ].format(DatabaseName, table_name, value["Name"], key)
        else:
            temp_middle_1 = "{} {}".format(
                parameters["ANALYSIS"]["CONTINUOUS"]["DISTRIBUTION"]["middle_1"],
                parameters["ANALYSIS"]["CONTINUOUS"]["DISTRIBUTION"]["bottom"].format(
                    DatabaseName, table_name, value["Name"], key
                ),
            )
            temp_middle_2 = parameters["ANALYSIS"]["CONTINUOUS"]["DISTRIBUTION"][
                "middle_2"
            ].format(value["Name"])

            if cont == size_continuous:

                table_top_var += "{} {}".format(
                    value["Name"],
                    parameters["ANALYSIS"]["CONTINUOUS"]["DISTRIBUTION"]["top_3"],
                )
                table_bottom += "{} {})".format(temp_middle_1, temp_middle_2)
            else:
                table_top_var += "{} ,".format(value["Name"])
                table_bottom += "{} {}".format(temp_middle_1, temp_middle_2)
        var_index += 1

query = (
    parameters["ANALYSIS"]["CONTINUOUS"]["DISTRIBUTION"]["top_1"]
    + table_top
    + parameters["ANALYSIS"]["CONTINUOUS"]["DISTRIBUTION"]["top_2"]
    + table_top_var
    + table_bottom
)
output = s3.run_query(
    query=query,
    database=DatabaseName,
    s3_output="SQL_OUTPUT_ATHENA",
    filename="count_distribution",  ## Add filename to print dataframe
    destination_key=None,  ### Add destination key if need to copy output
)

display(output.sort_values(by="pct").set_index(["pct"]).style.format("{0:.2f}"))

# Generation report

In [ ]:
import os, time, shutil, urllib, ipykernel, json
from pathlib import Path
from notebook import notebookapp

In [ ]:
def create_report(extension = "html", keep_code = False):
    """
    Create a report from the current notebook and save it in the 
    Report folder (Parent-> child directory)
    
    1. Exctract the current notbook name
    2. Convert the Notebook 
    3. Move the newly created report
    
    Args:
    extension: string. Can be "html", "pdf", "md"
    
    
    """
    
    ### Get notebook name
    connection_file = os.path.basename(ipykernel.get_connection_file())
    kernel_id = connection_file.split('-', 1)[0].split('.')[0]

    for srv in notebookapp.list_running_servers():
        try:
            if srv['token']=='' and not srv['password']:  
                req = urllib.request.urlopen(srv['url']+'api/sessions')
            else:
                req = urllib.request.urlopen(srv['url']+ \
                                             'api/sessions?token=' + \
                                             srv['token'])
            sessions = json.load(req)
            notebookname = sessions[0]['name']
        except:
            pass  
    
    sep = '.'
    path = os.getcwd()
    #parent_path = str(Path(path).parent)
    
    ### Path report
    #path_report = "{}/Reports".format(parent_path)
    #path_report = "{}/Reports".format(path)
    
    ### Path destination
    name_no_extension = notebookname.split(sep, 1)[0]
    source_to_move = name_no_extension +'.{}'.format(extension)
    dest = os.path.join(path,'Reports', source_to_move)
    
    ### Generate notebook
    if keep_code:
        os.system('jupyter nbconvert --to {} {}'.format(
    extension,notebookname))
    else:
        os.system('jupyter nbconvert --no-input --to {} {}'.format(
    extension,notebookname))
    
    ### Move notebook to report folder
    #time.sleep(5)
    shutil.move(source_to_move, dest)
    print("Report Available at this adress:\n {}".format(dest))

In [ ]:
create_report(extension = "html", keep_code = True)